In [188]:
import xarray as xr
import sqlite3
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd
import logging

# Open the GRIB file
print('opening GRIB file')
filename = "src/data/past_climate.grib"
variables = ['tp', 'tcc', 'rh', '2t','2d','10v', '10u'] 

datasets = {}
# Set up Dask to use a single thread
dask.config.set(scheduler='single-threaded')
for var in variables:
    try:
        # Open the GRIB file with chunks
        ds = xr.open_dataset(filename, engine='cfgrib', backend_kwargs={'filter_by_keys': {'shortName': var}}, chunks={'time': 10})
        datasets[var] = ds
    except Exception as e:
        logging.error('Error opening dataset for variable %s: %s', var, e)

datasets['2t']

opening GRIB file


<xarray.Dataset>
Dimensions:     (time: 8928, latitude: 9, longitude: 9)
Coordinates:
    number      int64 ...
  * time        (time) datetime64[ns] 1992-01-01 ... 2022-12-01T23:00:00
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 38.96 38.71 38.46 38.21 ... 37.46 37.21 36.96
  * longitude   (longitude) float64 -3.435 -3.185 -2.935 ... -1.934 -1.684 358.6
    valid_time  (time) datetime64[ns] dask.array<chunksize=(10,), meta=np.ndarray>
Data variables:
    t2m         (time, latitude, longitude) float32 dask.array<chunksize=(10, 9, 9), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-02-14T13:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [189]:
# Flatten the 'valid_time' coordinate from 2D to 1D
datasets['tp']['valid_time'] = datasets['tp']['valid_time'].values.flatten()

# Stack the 'time' and 'step' dimensions of the 'tp' variable into a single new dimension called 'total_time'
datasets['tp']['tp'] = datasets['tp']['tp'].stack(total_time=('time', 'step'))

# Assign the flattened 'valid_time' data to the 'total_time' coordinate
datasets['tp'].coords["valid_time"] = ("total_time", datasets['tp']['valid_time'].data)

# Swap the 'total_time' dimension with the 'valid_time' dimension
datasets['tp'] = datasets['tp'].swap_dims({'total_time': 'valid_time'})

# Drop the 'time', 'step', 'number', and 'surface' dimensions from 'tp_single_time_and_step' and assign the result to 'tp_dropped'
tp_dropped = datasets['tp'].drop(['time','step','number','surface'])

# Rename the 'valid_time' dimension to 'time' in 'tp_dropped' and assign the result back to 'datasets['tp']'
datasets['tp'] = tp_dropped.rename({'valid_time': 'time'})

# Drop all NA values in the 'time' dimension of 'datasets['tp']' and assign the result back to 'datasets['tp']'
datasets['tp'] = datasets['tp'].dropna(dim='time', how='all')

#TODO:check if the transpose works correctly
# Reorder the dimensions of 'datasets['tp']' to have 'time' as the first dimension
datasets['tp'] = datasets['tp'].transpose('time', 'latitude', 'longitude')

# Display the 'datasets['tp']' Dataset
datasets['tp']

<xarray.Dataset>
Dimensions:    (latitude: 9, longitude: 9, time: 8928)
Coordinates:
  * latitude   (latitude) float64 38.96 38.71 38.46 38.21 ... 37.46 37.21 36.96
  * longitude  (longitude) float64 -3.435 -3.185 -2.935 ... -1.934 -1.684 358.6
  * time       (time) datetime64[ns] 1992-01-01T01:00:00 ... 2022-12-02
Data variables:
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(78, 9, 9), meta=np.ndarray>
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-02-14T13:06 GRIB to CDM+CF via cfgrib-0.9.1...

In [ ]:
import sqlite3

# Connect to the database
conn = sqlite3.connect('data.db')

# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query to select all rows from the prediction_data table
cursor.execute('SELECT * FROM prediction_data')

# Fetch all rows from the result set
rows = cursor.fetchall()

# Print the rows
for row in rows:
    print(row)

# Close the cursor and connection
cursor.close()
conn.close()


In [183]:
datasets['tp']
tp_values = datasets['tp'].sel(latitude=38.96, longitude=-3.435, method = 'nearest')['tp'].values[0:36]
tp_values


array([8.0792461e-06, 1.2970982e-05, 3.8561407e-06, 2.6651196e-06,
       2.7853739e-06, 4.2487236e-06, 2.2627482e-06, 8.5088759e-06,
       3.4110963e-05, 3.3711789e-05, 2.5713187e-05, 1.4016936e-05,
       1.1954665e-05, 5.4037391e-06, 5.2015645e-07, 2.9290049e-07,
       1.7369121e-07, 1.0784839e-07, 1.6735066e-06, 2.3277180e-06,
       0.0000000e+00, 6.1363693e-07, 9.0918383e-07, 1.5360283e-06,
       6.3257175e-05, 5.7137135e-05, 1.0614641e-04, 7.1666567e-05,
       7.1574104e-05, 8.4010942e-05, 7.4455151e-05, 7.9221179e-05,
       8.0584708e-05, 9.0815825e-05, 9.7211640e-05, 8.9507696e-05],
      dtype=float32)

In [141]:

avg = datasets['tp'].sel(time='1994-10-01T18:00:00.000000000', latitude=38.96, longitude=-3.435, method = 'nearest')['tp'].mean(skipna=True).values
avg

array(5.003794e-05, dtype=float32)

In [169]:
datasets['tp']['time'].values[0:150]

array(['1991-12-31T19:00:00.000000000', '1991-12-31T20:00:00.000000000',
       '1991-12-31T21:00:00.000000000', '1991-12-31T22:00:00.000000000',
       '1991-12-31T23:00:00.000000000', '1992-01-01T00:00:00.000000000',
       '1992-01-01T01:00:00.000000000', '1992-01-01T02:00:00.000000000',
       '1992-01-01T03:00:00.000000000', '1992-01-01T04:00:00.000000000',
       '1992-01-01T05:00:00.000000000', '1992-01-01T06:00:00.000000000',
       '1992-01-01T07:00:00.000000000', '1992-01-01T08:00:00.000000000',
       '1992-01-01T09:00:00.000000000', '1992-01-01T10:00:00.000000000',
       '1992-01-01T11:00:00.000000000', '1992-01-01T12:00:00.000000000',
       '1992-01-01T13:00:00.000000000', '1992-01-01T14:00:00.000000000',
       '1992-01-01T15:00:00.000000000', '1992-01-01T16:00:00.000000000',
       '1992-01-01T17:00:00.000000000', '1992-01-01T18:00:00.000000000',
       '1992-01-01T19:00:00.000000000', '1992-01-01T20:00:00.000000000',
       '1992-01-01T21:00:00.000000000', '1992-01-01